In [1]:
import os, glob
import numpy as np
import pandas as pd
from scipy import stats
from datetime import date, time, datetime
from datetime import timedelta
from sklearn.model_selection import train_test_split
from keras.utils import np_utils
import keras
from keras.models import Sequential
from keras.layers import Dense,Dropout
from keras.utils import np_utils
import matplotlib
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix,  accuracy_score

w_list= glob.glob("../result/merge_weather/*.csv")
p_list = glob.glob("../result/flower_price_byweek/*")
select_t = [(1,0),(3,2),(2,1),(2,3),(2,4)]
st, flower = select_t[3]
fn_weather = w_list[st]
fn_price = p_list[flower]

print(fn_weather, fn_price)

Using TensorFlow backend.


../result/merge_weather\C0G650_員林.csv ../result/flower_price_byweek\OrientalLily_pbyweek.csv


In [2]:
r_data = pd.read_csv(fn_weather, encoding="utf-8")
lc_d = pd.read_csv("../lunar_celeb_date.csv", encoding="utf-8")


col_fix =  ['T.Max', 'T.Min', 'Precp','Temperature', 'RH', 'StnPres', 'WS', 'WSGust']
# rm "/"
for col in col_fix:
    old = r_data[col]
    new = []
    for i in range(len(old)):
        try:
            new.append(float(old[i]))
            tmp = float(old[i])
        except ValueError:
            new.append(tmp)
    r_data[col] = new
d_tmp = r_data['T.Max'] - r_data['T.Min']
r_data["d_tmp"]= d_tmp

# extract_date
d_data = r_data["Date"]
drop_c =["Date",'T.Max', 'T.Min']
r_data = r_data.drop(columns=drop_c)

def to_zscore2(df):
    col_x = df.columns.to_list()
    mean = df.mean(axis=0)
    std = df.std(axis=0)
    for i in range(len(col_x)):
        df[col_x[i]]=(df[col_x[i]]-mean[i])/std[i]
    return  df , (mean, std)

r_data, recordz = to_zscore2(r_data)
r_data["Date"] = d_data
r_data = r_data.join(lc_d.set_index("date"), on="Date")
r_data = r_data.drop(columns=["Date"])
r_data.head(10)

,Temperature,Precp,RH,StnPres,WS,WSGust,d_tmp,g_marry,g_pray,g_funeral,...,b_pray,b_funeral,b_business,month,spring,valetine_west,mother,graduation,ghost,valetine_east
0,-1.405393,-0.246367,-0.342541,0.782879,-1.540053,-1.307154,2.639062,1.0,1.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,-0.965367,-0.246367,-0.586510,0.512573,-0.997534,-1.398144,2.521348,0.0,1.0,0.0,...,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2,-0.585346,-0.179558,-0.220556,0.191585,-0.726275,-1.170668,0.520201,0.0,1.0,0.0,...,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3,-1.165379,-0.246367,-1.074449,0.343632,-0.726275,-1.170668,0.912583,0.0,0.0,0.0,...,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,-1.565402,-0.246367,-1.562388,0.664620,-0.997534,-1.352649,1.422679,0.0,1.0,1.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
5,-1.625405,-0.246367,-0.342541,0.749091,-0.726275,-1.307154,0.912583,0.0,1.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
6,-1.165379,-0.246367,-0.220556,0.613938,-0.997534,-1.079678,2.011252,0.0,0.0,1.0,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
7,-1.045372,-0.246367,-0.342541,0.647726,-0.455016,-1.079678,1.618870,1.0,0.0,0.0,...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
8,-1.705410,-0.246367,-1.074449,1.340385,0.901281,-0.351756,-0.892373,0.0,1.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
9,-1.525400,-0.246367,-0.586510,1.644479,-0.455016,-0.715717,0.363248,1.0,1.0,0.0,...,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [3]:
# print(r_data.shape)
# print(d_data.shape)

In [4]:
shift1=28
shift2=1

dfs =[]
arr_all =[]
if shift2==0:
    df_s = r_data.copy()
else:
    df_s = r_data.copy()
    df_s = df_s.shift(periods=shift2)
#     d_data = np.array(pd.DataFrame(d_data).shift(periods=shift2))
#     add shift base
arr_all = np.array(df_s)

for i in range(1,shift1):
    tp = np.array(df_s.shift(periods=i))
    arr_all = np.concatenate((arr_all, tp), axis=1)
df_all = pd.DataFrame(arr_all)
df_all["date"] = d_data
df_all = df_all.dropna()
df_all = df_all.reset_index()
df_all = df_all.drop(columns="index")

# r_data = df_all

# df_all.shape

In [5]:
r_data = df_all

d_data = r_data["date"]

def trans_to_y_w(d_date):
    year = int(d_date.split("-")[0])
    d_day = date(year,int(d_date.split("-")[1]), int(d_date.split("-")[2]))- date(year, 1, 1)
    d_w = 1+ (d_day.days // 7)
    if d_w ==53:
        d_w = 52
    result =  str(year) + "_" + str(d_w)

    if d_day.days % 7 ==6:
        return result
    else:
        return np.nan
    
d_new = []
for i in range(len(d_data)):
    d_new.append(trans_to_y_w(d_data[i]))
r_data["y_w"] = d_new
r_data = r_data.dropna()
r_data = r_data.reset_index()
r_data = r_data.drop(columns="index")
# r_data
# d_data.shape

In [6]:
df_pbw = pd.read_csv(fn_price, encoding="utf-8")
# join x and y
df_join = df_pbw.join(r_data.set_index("y_w"), on="y_w")
df_join = df_join.dropna()
df_join = df_join.reset_index()
df_join = df_join.drop(columns="index")
# extract y info
y_date = df_join["date"]
y_yw = df_join["y_w"]
y_raw = np.array(df_join["price_diff"])


In [7]:

d_t_dummy = ["market"]
tmp_dum = pd.DataFrame()
for col in d_t_dummy:
    tmp_dum[col] = df_join[col]
    dummy = pd.get_dummies(tmp_dum[col])
    df_join = pd.concat([df_join, dummy], axis=1)
drop_c = ["market","year", "week","w_avg", "w_sale", "date", "y_w", "price_diff"]
# x finished
x = np.array(df_join.drop(columns=drop_c))
# x
print(x.shape)

(7843, 621)


In [8]:
# modify y
# y_raw = np.array(df_join["price_diff"])
print("y_price_diff describe:")
print(pd.DataFrame(y_raw, columns=["y_price_diff"]).describe())
print("======")
def y_to_class(v):
    t = 0
    y_class_range = [-0.1,0,0.1]
#     for q in range(1,12):
#         y_class_range.append(round((q*0.1-0.6),4))

    for i in range(len(y_class_range)):
        if (v >= y_class_range[i]):
            t = i+1
    return int(t)

y_class = []

for i in range(len(y_raw)):
    y_class.append(y_to_class(y_raw[i]))
print("freq:",np.bincount(y_class))

y = np.array(y_class)
input_d = x.shape[1]
print(x.shape,len(y))
# x y ready

y_price_diff describe:
       y_price_diff
count   7843.000000
mean       0.028905
std        0.266343
min       -0.723768
25%       -0.145503
50%        0.001067
75%        0.169860
max        1.566871
freq: [2568 1347 1358 2570]
(7843, 621) 7843


In [9]:
# split data
# y_date
# x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.1)
y_year = []
for i in range(len(y_yw)):
    y_year.append(int(y_yw[i].split("_")[0]))
y_year = np.array(y_year)
years, count = np.unique(np.array(y_year), return_counts=True)
print(years)
print(count)

train = y_year[:]<=2017
test = y_year[:]>2017

x_train, y_train = x[train,:], y[train]
x_test, y_test = x[test,:], y[test]


[2014 2015 2016 2017 2018]
[1457 1581 1581 1612 1612]


In [10]:
out_class_count = len(np.bincount(y))
# shifts = 28
result_table = []
max_res = []
acc_cf =0
# dinp, epochs = (7,40)
for dinp in [2,4,7,9]:
    for epochs in [2,4,6]:
        print("dinp, epochs:",dinp, epochs)
        #     epochs = 80
        # xy_data = (x_train, y_train, x_test, y_test)
        # x_train, y_train, x_test, y_test = xy_data
        model = Sequential()
        layer_0 = Dense(units =input_d//dinp, input_dim = input_d,
                        kernel_initializer = "random_normal", activation = "relu")
        model.add(layer_0)
        model.add(Dropout(0.25))
        layer_1 = Dense(units =(shift1+1),kernel_initializer = "random_normal", activation = "relu")
        model.add(layer_1)
        model.add(Dropout(0.25))
        layer_out = Dense(units = out_class_count,kernel_initializer = "random_normal", 
                          activation = "softmax")
        model.add(layer_out)
        #     model.summary()
        model.compile(loss="categorical_crossentropy",optimizer = "adam", metrics = ['accuracy'])
        train_history = model.fit(x = x_train, y = np_utils.to_categorical(y_train), 
                                  validation_split = 0.2, epochs =epochs, verbose = 0)
        #     plt.plot(train_history.history["loss"])
        #     plt.plot(train_history.history["val_loss"])
        #     plt.title("Loss Graph")
        #     plt.legend(['loss', 'val_loss'], loc="upper left")

        #     model_test
        pre = model.predict_classes(x_test)
        acc = round(accuracy_score(y_test, pre)*100,2)
        #         if acc > acc_cf:
        #             max_res = dinp, epochs, acc
        #             acc_cf = acc
        #         result_table.append([dinp, epochs, acc])
        print(acc)

dinp, epochs: 2 2
38.46
dinp, epochs: 2 4
34.24
dinp, epochs: 2 6
34.74
dinp, epochs: 4 2
35.3
dinp, epochs: 4 4
41.75
dinp, epochs: 4 6
31.02
dinp, epochs: 7 2
33.25
dinp, epochs: 7 4
32.32
dinp, epochs: 7 6
31.14
dinp, epochs: 9 2
35.67
dinp, epochs: 9 4
36.66
dinp, epochs: 9 6
35.79


In [11]:
# out_class_count = len(np.bincount(y))
# shifts = 28
# xy_data = (x_train, y_train, x_test, y_test)
# def model_build(xy_data, input_d, shifts, epochs=60):
#     x_train, y_train, x_test, y_test = xy_data
    
#     model = Sequential()
#     layer_0 = Dense(units =input_d//4, input_dim = input_d,
#                     kernel_initializer = "random_normal", activation = "relu")
#     model.add(layer_0)
#     model.add(Dropout(0.25))
#     layer_1 = Dense(units =shifts,kernel_initializer = "random_normal", activation = "relu")
#     model.add(layer_1)
#     model.add(Dropout(0.25))
#     layer_out = Dense(units = out_class_count,kernel_initializer = "random_normal", 
#                       activation = "softmax")
#     model.add(layer_out)
#     #     model.summary()
#     model.compile(loss="categorical_crossentropy",optimizer = "adam", metrics = ['accuracy'])
#     train_history = model.fit(x = x_train, y = np_utils.to_categorical(y_train), 
#                               validation_split = 0.1, epochs =epochs, verbose = 2)
    
#     #     model_test
#     pre = model.predict_classes(x_test)
#     acc = round(accuracy_score(y_test, pre)*100,2)
    
#     return acc, model, train_history

# acc_cf = 0

# for epochs in range(5,80,5):
#     print("epochs:", epochs)
#     acc, model ,train_history= model_build(xy_data, input_d, shifts, epochs=60)
#     print(train_history)
#     print(acc)
#     if acc > acc_cf:
#         acc_cf = acc
#         bestmodel = model



In [12]:
# acc
# pd.DataFrame(result_table)
# max_res

In [13]:
# from sklearn.metrics import confusion_matrix
# pre = model.predict_classes(x_test)
# pd.DataFrame(confusion_matrix(y_test, pre))

In [14]:
# x_tr, y_tr = x[train,:], y_raw[train]
# x_te, y_te = x[test,:], y_raw[test]
# dinp, epochs = (10,400)
# model = Sequential()
# layer_0 = Dense(units =input_d//dinp, input_dim = input_d,
#                 kernel_initializer = "random_normal", activation = "relu")
# model.add(layer_0)
# model.add(Dropout(0.25))

# layer_0 = Dense(units =input_d//(dinp*5), input_dim = input_d,
#                 kernel_initializer = "random_normal", activation = "relu")
# model.add(layer_0)
# model.add(Dropout(0.25))

# layer_out = Dense(units = 1,kernel_initializer = "random_normal", 
#                   activation = "linear")
# model.add(layer_out)
# model.summary()
# model.compile(loss="mse",optimizer = "adam")
# train_history = model.fit(x = x_tr, y = y_tr, 
#                           validation_split = 0.2, epochs =epochs, verbose = 0)
# plt.plot(train_history.history["loss"])
# plt.plot(train_history.history["val_loss"])
# plt.title("Loss Graph")
# plt.legend(['loss', 'val_loss'], loc="upper left")



In [15]:
# rs = []
# pre = model.predict(x_te)
# for i in range(len(pre)):
#     rs.append([pre[i][0], y_te[i]])
# df_rs = pd.DataFrame(rs, columns=["pre", "ans"])
# # df_rs